In [91]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import zscore

# strategy:
## using overlapping cultivar lines


In [3]:
## read accession lines which present in duplicates in phenotyped lines
df = pd.read_excel("accession_info.xlsx",sheet_name='duplicates',header=0,
                   converters={'Plot':str})
df.head()

,ACCNAME,SET,Plot,Category
0,Almaz,cultivar,1001,cultivar
1,CBA Captain,cultivar,1002,cultivar
2,Genesis 090,cultivar,1003,cultivar
3,Genesis 836,cultivar,1004,cultivar
4,Genesis Kalkee,cultivar,1005,cultivar


In [4]:
df['ACCNAME'] = df['ACCNAME'].apply(str.lower)
df.head()

,ACCNAME,SET,Plot,Category
0,almaz,cultivar,1001,cultivar
1,cba captain,cultivar,1002,cultivar
2,genesis 090,cultivar,1003,cultivar
3,genesis 836,cultivar,1004,cultivar
4,genesis kalkee,cultivar,1005,cultivar


In [5]:
## read phenotype and merged with genoID
df2 = pd.read_csv("df_merged.csv",header=0)
df2.head()

,ID,Category,Group,Num,mean_T0,mean_T1,RTI,geno_id,geno,plot_id,category
0,33005,HTRCxCDB22C,G1,1,70.000000,87.333333,1.247619,Pulse.30K-0043-06-81|33005-CH22.HTRCxCDB22C,Pulse.30K-0043-06-81,33005.0,CH22.HTRCxCDB22C
1,33010,HTRCxCDB22C,G1,10,92.666667,110.666667,1.194245,Pulse.30K-0043-06-86|33010-CH22.HTRCxCDB22C,Pulse.30K-0043-06-86,33010.0,CH22.HTRCxCDB22C
2,35002,HTRCxCDB22C,G1,100,84.333333,75.000000,0.889328,Pulse.30K-0043-07-17|35002-CH22.HTRCxCDB22C,Pulse.30K-0043-07-17,35002.0,CH22.HTRCxCDB22C
3,35001,HTRCxCDB22C,G1,101,73.000000,104.666667,1.433790,Pulse.30K-0043-07-16|35001-CH22.HTRCxCDB22C,Pulse.30K-0043-07-16,35001.0,CH22.HTRCxCDB22C
4,32013,HTRCxCDB22C,G1,102,80.333333,72.333333,0.900415,NaN,NaN,NaN,NaN


In [78]:
df2.shape

(1119, 11)

In [6]:
df_merged = df.merge(df2,how='left',left_on='Plot',right_on='ID')
df_merged.head()

,ACCNAME,SET,Plot,Category_x,ID,Category_y,Group,Num,mean_T0,mean_T1,RTI,geno_id,geno,plot_id,category
0,almaz,cultivar,1001,cultivar,1001,cultivar,G6,169.0,112.333333,34.666667,0.308605,Pulse.30K-0043-20-02|1001-Almaz,Pulse.30K-0043-20-02,1001.0,cultivar
1,cba captain,cultivar,1002,cultivar,1002,cultivar,G4,75.0,160.333333,44.666667,0.278586,Pulse.30K-0043-20-03|1002-CBA-Captain,Pulse.30K-0043-20-03,1002.0,cultivar
2,genesis 090,cultivar,1003,cultivar,1003,cultivar,G4,74.0,166.666667,54.666667,0.328000,Pulse.30K-0043-20-04|1003-Genesis-090,Pulse.30K-0043-20-04,1003.0,cultivar
3,genesis 836,cultivar,1004,cultivar,1004,cultivar,G4,73.0,119.000000,38.666667,0.324930,Pulse.30K-0043-20-05|1004-Genesis-836,Pulse.30K-0043-20-05,1004.0,cultivar
4,genesis kalkee,cultivar,1005,cultivar,1005,cultivar,G4,72.0,161.333333,59.000000,0.365702,Pulse.30K-0043-20-06|1005-Genesis-Kalkee,Pulse.30K-0043-20-06,1005.0,cultivar


In [9]:
## generate all pairs for 6 groups
from itertools import combinations

In [10]:
group_list = ['G1','G2','G3','G4','G5','G6']
res = list(combinations(group_list, 2))
res

[('G1', 'G2'),
 ('G1', 'G3'),
 ('G1', 'G4'),
 ('G1', 'G5'),
 ('G1', 'G6'),
 ('G2', 'G3'),
 ('G2', 'G4'),
 ('G2', 'G5'),
 ('G2', 'G6'),
 ('G3', 'G4'),
 ('G3', 'G5'),
 ('G3', 'G6'),
 ('G4', 'G5'),
 ('G4', 'G6'),
 ('G5', 'G6')]

In [12]:
## get ACCNAME for each group and put in a dict
group_dict = {}
for group in group_list:
    ID = df_merged[df_merged['Group'] == group]['ACCNAME'].to_list()
    UID = list(set(ID))
    print(group, len(ID),len(UID))
    group_dict[group] = UID
    

G1 11 11
G2 23 17
G3 0 0
G4 70 42
G5 26 25
G6 69 49


In [14]:
## count overlap between group pairs
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

for pair in res:
    I = pair[0]
    J = pair[1]
    OVERLAP = intersection(group_dict[I],group_dict[J])
    print(I,J, len(OVERLAP))

G1 G2 8
G1 G3 0
G1 G4 11
G1 G5 1
G1 G6 11
G2 G3 0
G2 G4 13
G2 G5 1
G2 G6 16
G3 G4 0
G3 G5 0
G3 G6 0
G4 G5 5
G4 G6 16
G5 G6 20


In [40]:
## all group compared to G1, leave out G3 which has no overlap with others
# Step1: groupby for each group to account for replicates in each group
# Step2: plot overlapping lines between groups, check for consistence in overlapping lines
# Step3: calculate ratio between groups
# Step4: normalize value based on average ratio
cols = ['ACCNAME','Group','mean_T0','mean_T1','RTI']

df_G1 = df_merged[df_merged['Group'] == 'G1'][cols].groupby(['ACCNAME','Group']).mean().reset_index()
df_G2 = df_merged[df_merged['Group'] == 'G2'][cols].groupby(['ACCNAME','Group']).mean().reset_index()
df_G3 = df_merged[df_merged['Group'] == 'G3'][cols].groupby(['ACCNAME','Group']).mean().reset_index()
df_G4 = df_merged[df_merged['Group'] == 'G4'][cols].groupby(['ACCNAME','Group']).mean().reset_index()
df_G5 = df_merged[df_merged['Group'] == 'G5'][cols].groupby(['ACCNAME','Group']).mean().reset_index()
df_G6 = df_merged[df_merged['Group'] == 'G6'][cols].groupby(['ACCNAME','Group']).mean().reset_index()

## compare to G1
df_G1G2 = df_G1.merge(df_G2,on='ACCNAME',how='inner')
df_G1G4 = df_G1.merge(df_G4,on='ACCNAME',how='inner')
df_G1G5 = df_G1.merge(df_G5,on='ACCNAME',how='inner')
df_G1G6 = df_G1.merge(df_G6,on='ACCNAME',how='inner')

## or compare to G6, more overlaps
df_G6G1 = df_G6.merge(df_G1,on='ACCNAME',how='inner')
df_G6G2 = df_G6.merge(df_G2,on='ACCNAME',how='inner')
df_G6G4 = df_G6.merge(df_G4,on='ACCNAME',how='inner')
df_G6G5 = df_G6.merge(df_G5,on='ACCNAME',how='inner')

In [79]:
df2.head(2)

,ID,Category,Group,Num,mean_T0,mean_T1,RTI,geno_id,geno,plot_id,category
0,33005,HTRCxCDB22C,G1,1,70.000000,87.333333,1.247619,Pulse.30K-0043-06-81|33005-CH22.HTRCxCDB22C,Pulse.30K-0043-06-81,33005.0,CH22.HTRCxCDB22C
1,33010,HTRCxCDB22C,G1,10,92.666667,110.666667,1.194245,Pulse.30K-0043-06-86|33010-CH22.HTRCxCDB22C,Pulse.30K-0043-06-86,33010.0,CH22.HTRCxCDB22C


## zscore after adjustment

In [85]:
## compare with the RTI scater plot, the ratio calculated by G6 is generally consistent
## for G3, just use scalling

def cal_ratio(DF1,Trait):
    Y = Trait + "_y"
    X = Trait + "_x"
    ratios = DF1[Y] / DF1[X]
    return ratios.mean(), ratios.std()

# get group phenotype, but keep the replicates
cols2 = ['geno_id','Group','mean_T0','mean_T1','RTI'] 
dfG1 = df2[df2['Group'] == 'G1'][cols2]
dfG2 = df2[df2['Group'] == 'G2'][cols2]
dfG3 = df2[df2['Group'] == 'G3'][cols2]
dfG4 = df2[df2['Group'] == 'G4'][cols2]
dfG5 = df2[df2['Group'] == 'G5'][cols2]
dfG6 = df2[df2['Group'] == 'G6'][cols2]

# adjust G1, G2, G5 to G6
for TRAIT in ['mean_T0','mean_T1','RTI']:
    dfG1[TRAIT] = dfG1[TRAIT] / cal_ratio(df_G6G1, TRAIT)[0]
    dfG2[TRAIT] = dfG2[TRAIT] / cal_ratio(df_G6G2, TRAIT)[0]
    dfG4[TRAIT] = dfG4[TRAIT] / cal_ratio(df_G6G4, TRAIT)[0]
    dfG5[TRAIT] = dfG5[TRAIT] / cal_ratio(df_G6G5, TRAIT)[0]

    ## z-score normalization
    dfG1[TRAIT] = stats.zscore(dfG1[TRAIT])
    dfG2[TRAIT] = stats.zscore(dfG2[TRAIT])
    dfG3[TRAIT] = stats.zscore(dfG3[TRAIT])
    dfG4[TRAIT] = stats.zscore(dfG4[TRAIT])
    dfG5[TRAIT] = stats.zscore(dfG5[TRAIT])
    dfG6[TRAIT] = stats.zscore(dfG6[TRAIT])

## merge adjusted value
df_adjust = pd.concat([dfG1,dfG2,dfG3,dfG4,dfG5,dfG6]).dropna().reset_index(drop=True)

In [86]:
df_adjust.shape

(974, 5)

In [87]:
df_adjust.head()

,geno_id,Group,mean_T0,mean_T1,RTI
0,Pulse.30K-0043-06-81|33005-CH22.HTRCxCDB22C,G1,-0.923516,-0.616678,0.201794
1,Pulse.30K-0043-06-86|33010-CH22.HTRCxCDB22C,G1,-0.085422,0.306900,0.083517
2,Pulse.30K-0043-07-17|35002-CH22.HTRCxCDB22C,G1,-0.393545,-1.104855,-0.592168
3,Pulse.30K-0043-07-16|35001-CH22.HTRCxCDB22C,G1,-0.812591,0.069409,0.614342
4,Pulse.30K-0043-07-15|34016-CH22.HTRCxCDB22C,G1,-0.147047,0.372870,0.172572


In [88]:
## output phenotype for groups
cols = ['geno_id','mean_T0','mean_T1','RTI']
for group in df_adjust["Group"].unique():
    df_adjust[df_adjust["Group"] == group][cols].to_csv(group + '_zscore.phenotype', sep='\t',index=False)

In [89]:
## output phenotype for All group
df_adjust[cols].to_csv('All_zscore.phenotype', sep='\t',index=False)

## zscore/log from all but G3, not individual as above

In [95]:
df2.head()

,ID,Category,Group,Num,mean_T0,mean_T1,RTI,geno_id,geno,plot_id,category
0,33005,HTRCxCDB22C,G1,1,70.000000,87.333333,1.247619,Pulse.30K-0043-06-81|33005-CH22.HTRCxCDB22C,Pulse.30K-0043-06-81,33005.0,CH22.HTRCxCDB22C
1,33010,HTRCxCDB22C,G1,10,92.666667,110.666667,1.194245,Pulse.30K-0043-06-86|33010-CH22.HTRCxCDB22C,Pulse.30K-0043-06-86,33010.0,CH22.HTRCxCDB22C
2,35002,HTRCxCDB22C,G1,100,84.333333,75.000000,0.889328,Pulse.30K-0043-07-17|35002-CH22.HTRCxCDB22C,Pulse.30K-0043-07-17,35002.0,CH22.HTRCxCDB22C
3,35001,HTRCxCDB22C,G1,101,73.000000,104.666667,1.433790,Pulse.30K-0043-07-16|35001-CH22.HTRCxCDB22C,Pulse.30K-0043-07-16,35001.0,CH22.HTRCxCDB22C
4,32013,HTRCxCDB22C,G1,102,80.333333,72.333333,0.900415,NaN,NaN,NaN,NaN


In [100]:
## compare with the RTI scater plot, the ratio calculated by G6 is generally consistent
## for G3, just use scalling

def cal_ratio(DF1,Trait):
    Y = Trait + "_y"
    X = Trait + "_x"
    ratios = DF1[Y] / DF1[X]
    return ratios.mean(), ratios.std()

# get group phenotype, but keep the replicates
cols2 = ['geno_id','ID','Category','Group','mean_T0','mean_T1','RTI'] 
dfG1 = df2[df2['Group'] == 'G1'][cols2]
dfG2 = df2[df2['Group'] == 'G2'][cols2]
dfG3 = df2[df2['Group'] == 'G3'][cols2]
dfG4 = df2[df2['Group'] == 'G4'][cols2]
dfG5 = df2[df2['Group'] == 'G5'][cols2]
dfG6 = df2[df2['Group'] == 'G6'][cols2]

# adjust G1, G2, G5 to G6
for TRAIT in ['mean_T0','mean_T1','RTI']:
    dfG1[TRAIT] = dfG1[TRAIT] / cal_ratio(df_G6G1, TRAIT)[0]
    dfG2[TRAIT] = dfG2[TRAIT] / cal_ratio(df_G6G2, TRAIT)[0]
    dfG4[TRAIT] = dfG4[TRAIT] / cal_ratio(df_G6G4, TRAIT)[0]
    dfG5[TRAIT] = dfG5[TRAIT] / cal_ratio(df_G6G5, TRAIT)[0]

## merge adjusted value, remove G3 for test
df_adjust = pd.concat([dfG1,dfG2,dfG4,dfG5,dfG6]).dropna().reset_index(drop=True)


## zscore/log normalization for all
for TRAIT in ['mean_T0','mean_T1','RTI']:
    #df_adjust[TRAIT] = stats.zscore(df_adjust[TRAIT])
    #df_adjust[TRAIT] = np.log(df_adjust[TRAIT])
    df_adjust[TRAIT] = stats.zscore(df_adjust[TRAIT])
    dfG3[TRAIT] = stats.zscore(dfG3[TRAIT])

## output phenotype for All group
cols = ['geno_id','mean_T0','mean_T1','RTI']
#df_adjust[cols].to_csv('noG3_zscore.phenotype', sep='\t',index=False)
#df_adjust[cols].to_csv('noG3_log.phenotype', sep='\t',index=False)

df_adjust2 = pd.concat([df_adjust,dfG3]).dropna().reset_index(drop=True)
df_adjust2.to_csv('all_zscore_plotting.phenotype', sep='\t',index=False)

#df_adjust[df_adjust['Category'] == 'HTRCxCDB22C'][cols].to_csv('HTRCxCDB22C_noG3_zscore.phenotype', sep='\t',index=False)

In [94]:
df_adjust.head()

,geno_id,Group,mean_T0,mean_T1,RTI
0,Pulse.30K-0043-06-81|33005-CH22.HTRCxCDB22C,G1,-1.235003,-0.713519,0.391322
1,Pulse.30K-0043-06-86|33010-CH22.HTRCxCDB22C,G1,-0.521683,-0.062180,0.257875
2,Pulse.30K-0043-07-17|35002-CH22.HTRCxCDB22C,G1,-0.783933,-1.057798,-0.504481
3,Pulse.30K-0043-07-16|35001-CH22.HTRCxCDB22C,G1,-1.140593,-0.229667,0.856789
4,Pulse.30K-0043-07-15|34016-CH22.HTRCxCDB22C,G1,-0.574133,-0.015656,0.358352


In [17]:
## count the number of group that contain each line
line_dict = {}
for LINE in df_merged['ACCNAME'].unique():
    GROUPS = df_merged[df_merged['ACCNAME'] == LINE]['Group'].unique()
    COUNT = len(GROUPS)
    line_dict[LINE] = COUNT

In [23]:
df_line_count = pd.DataFrame.from_dict(line_dict,orient='index')
df_line_count.columns=['group_count']
df_line_count = df_line_count.sort_values(by='group_count',ascending=False)
df_line_count.head()

,group_count
kyabra,5
pba maiden,4
pba pistol,4
cba captain,4
genesis 836,4
